<a href="https://www.kaggle.com/code/hossamhamza/resnet50v1-0-0?scriptVersionId=144433777" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#imports
import time
import torch
import torch.nn as nn #neural network module
import torch.optim as optim #optimizers (e.g. Gradient Descent Stochastic Gradient Descent (SGD))
import torch.nn.functional as F # contains activation functions
from torch.utils.data import DataLoader # to load data set
import torchvision.datasets as datasets # to download dataset form mnisc
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder



In [ ]:
def timer(f, txt="time"):
    def wrapper(*args):
        tic = time.time()
        val = f(*args)
        tac = time.time() - tic

        print(txt, tac, "ms")
        return val
    return wrapper

In [ ]:
# Applying Transforms to the Data
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}



#Setup Gdrive file download extention 
# !pip install gdown

In [ ]:
# set device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# import gdown 
# url = 'https://drive.google.com/uc?id=1hBzNW2XOp41v58GPTDITJvN_xrug1U8b' 
# output = 'drive.zip'
# gdown.download(url, output)

# !conda install -y gdown 
# !gdown --id 1hBzNW2XOp41v58GPTDITJvN_xrug1U8b
# ! unzip drive.zip
# ! rm drive.zip

# !python3 -m gdown --id 1hBzNW2XOp41v58GPTDITJvN_xrug1U8b



# Set train and valid directory paths
# train_directory = "/kaggle/input/monuments-v0/monuments/monuments/train"
# test_directory = "/kaggle/input/monuments-v0/monuments/monuments/test"
# data_directory = "/kaggle/input/eg-landmarks/images"
data_directory = "/kaggle/input/kgl-dtst/kaggle"

# valid_directory = "/kaggle/input/monuments-v0/monuments/monuments/valid"

# Batch size
bs = 32


# how many times the entire dataset is fed to the network
epochs = 8


# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=data_directory, transform=image_transforms['train']),
    'valid': True,
    'test': True
}

# Size of Data, to be used for calculating Average Loss and Accuracy
# test_data2 = DataLoader(datasets.ImageFolder(root=test2_directory, transform=image_transforms['test']), batch_size=bs, shuffle=True)
# test_data_size2 = len(datasets.ImageFolder(root=test2_directory, transform=image_transforms['test']))
class_names = data['train'].classes

# Number of classes
num_classes = len(class_names) #5

# train_data_size = len(data['train'])
# valid_data_size = len(data['valid'])
# test_data_size = len(data['test'])


# Create iterators for the Data loaded using DataLoader module
data['train'], data['test'], data['valid'] = torch.utils.data.random_split(data['train'], [0.7, 0.15, 0.15], generator=torch.Generator().manual_seed(42))
train_data_size = len(data['train'])
valid_data_size = len(data['test'])
test_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=bs, shuffle=True)
test_data = DataLoader(data['test'], batch_size=bs, shuffle=True)

# Print the train, validation and test set data sizes
train_data_size, valid_data_size, test_data_size

In [ ]:
# Load pretrained ResNet50 Model
resnet50 = models.resnet50(pretrained=True) # True to allow learning transfer


# Freeze model parameters
for param in resnet50.parameters():
    param.requires_grad = False


# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), 
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)


# Convert model to be used on GPU
resnet50 = resnet50.to(device)


# Define Optimizer and Loss Function
loss_func = nn.NLLLoss()
optimizer = optim.Adam(resnet50.parameters())
history = []




def train():
    for epoch in range(epochs):
            epoch_start = time.time()
            print("Epoch: {}/{}".format(epoch+1, epochs))
            # Set to training mode
            resnet50.train()
            # Loss and Accuracy within the epoch
            train_loss = 0.0
            train_acc = 0.0
            valid_loss = 0.0
            valid_acc = 0.0
            for i, (inputs, labels) in enumerate(train_data):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # Clean existing gradients
                optimizer.zero_grad()
                # Forward pass - compute outputs on input data using the model
                outputs = resnet50(inputs)
                # Compute loss
                loss = loss_func(outputs, labels)
                # Backpropagate the gradients
                loss.backward()
                # Update the parameters
                optimizer.step()
                # Compute the total loss for the batch and add it to train_loss
                train_loss += loss.item() * inputs.size(0)
                # Compute the accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))
                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))
                # Compute total accuracy in the whole batch and add to train_acc
                train_acc += acc.item() * inputs.size(0)
                print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))
    # Validation - No gradient tracking needed
            with torch.no_grad():
                # Set to evaluation mode
                resnet50.eval()
                # Validation loop
                for j, (inputs, labels) in enumerate(valid_data):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    # Forward pass - compute outputs on input data using the model
                    outputs = resnet50(inputs)
                    # Compute loss
                    loss = loss_func(outputs, labels)
                    # Compute the total loss for the batch and add it to valid_loss
                    valid_loss += loss.item() * inputs.size(0)
                    # Calculate validation accuracy
                    ret, predictions = torch.max(outputs.data, 1)
                    correct_counts = predictions.eq(labels.data.view_as(predictions))
                    # Convert correct_counts to float and then compute the mean
                    acc = torch.mean(correct_counts.type(torch.FloatTensor))
                    # Compute total accuracy in the whole batch and add to valid_acc
                    valid_acc += acc.item() * inputs.size(0)
                    print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            # Find average training loss and training accuracy
            avg_train_loss = train_loss/train_data_size 
            avg_train_acc = train_acc/float(train_data_size)
            # Find average training loss and training accuracy
            avg_valid_loss = valid_loss/valid_data_size 
            avg_valid_acc = valid_acc/float(valid_data_size)
            history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
            epoch_end = time.time()
            print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, nttValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

In [ ]:
def chech_accuracy(loader, model):
    num_correct=0
    num_samples=0
    model.eval()

#  This method disables the gradient calculation which reduces the memory consumption for computations.
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            
            
            num_correct += (y == predictions).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)}')
    



def predict(model, test_image_name):
    transform = image_transforms['test']
    test_image = Image.open(test_image_name)
    plt.imshow(test_image)
    test_image_tensor = transform(test_image)
    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
        print("Output class :  ", idx_to_class[topclass.cpu().numpy()[0][0]])



In [ ]:
params_path = "/kaggle/working/params"


train()
torch.save(resnet50.state_dict(), params_path)
# resnet50.load_state_dict(torch.load(params_path))

print(class_names)
chech_accuracy(test_data, resnet50)
# chech_accuracy(valid_data, resnet50)

